In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import cpca
import os
os.chdir("/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据")



df = pd.read_stata("final_panel_newweibo.dta")
df = df.sort_values(by=['adcode', 'date']).reset_index()
df["day"] = df.groupby("adcode").cumcount()
df['event'] = 0

# Identify events (first protest and subsequent protests >= 3 months apart)
for ad, ad_df in df.groupby('adcode'):
    protest_dates = ad_df.loc[ad_df['n_prt_rfa'] > 0, 'day'].sort_values().tolist()
    last_event = None
    
    for protest_date in protest_dates:
        if last_event is None or (protest_date - last_event) >= 45:
            df.loc[(df['adcode'] == int(ad)) & (df['day'] == protest_date),'event'] = 1
            last_event = protest_date

index_list = df.loc[df['event']==1,'index'].tolist()
df['to_day']=None
for index in index_list:
    for i in range(-22,23):
        if not (index+i<0) or (index+i>len(df)):
            df.loc[df['index']==index+i,'to_day'] = i

df.to_csv('eventstudy_rfa_county.csv')

